# Maji Mdogo

In [ ]:
import os
import re


In [ ]:
from data_ingestion import create_db_engine, query_to_dataframe, read_csv_web

sql_query = """
    SELECT * 
    FROM geographic_features
    LEFT JOIN weather_features USING(Field_ID)
    LEFT JOIN soil_and_crop_features USING(Field_ID)
    LEFT JOIN farm_management_features USING(Field_ID)
"""

weather_url= "https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_station_data.csv"
weather_mapping_url = "https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_data_field_mapping.csv"

field_df = query_to_dataframe(create_db_engine('Maji_Ndogo_farm_survey_small.db'), sql_query)
weather_df = read_csv_web(weather_url)
weather_mapping_df = read_csv_web(weather_mapping_url)

field_test = field_df.shape
weather_test = weather_df.shape
weather_mapping_test = weather_mapping_df.shape
print(f"field_df: {field_test}, weather_df: {weather_test}, weather_mapping_df: {weather_mapping_test}")


2025-06-08 22:59:31,678 - INFO - Database engine created successfully for Maji_Ndogo_farm_survey_small.db
2025-06-08 22:59:31,769 - INFO - Query executed successfully, returned 5654 rows.
2025-06-08 22:59:32,497 - INFO - CSV file read successfully from https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_station_data.csv, returned 1843 rows.
2025-06-08 22:59:33,162 - INFO - CSV file read successfully from https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_data_field_mapping.csv, returned 5654 rows.


field_df: (5654, 18), weather_df: (1843, 2), weather_mapping_df: (5654, 3)


In [3]:
# Import weather station data and weather field mapping data from their respective CSV files
weather_station_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_station_data.csv')
weather_station_mapping_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_data_field_mapping.csv')

### Data Cleanup

In [5]:
weather_station_mapping_df.head()

,Unnamed: 0,Field_ID,Weather_station
0,1003,40413,0
1,3638,4086,0
2,1186,32366,0
3,3644,16996,0
4,3647,16398,0


In [ ]:

# Swap the 'Crop_type' and 'Annual_yield' columns
MD_agric_df = MD_agric_df.rename(columns={'Crop_type': 'Annual_yield', 'Annual_yield': 'Crop_type'})

MD_agric_df.head()

,Field_ID,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Annual_yield,Crop_type,Standard_yield
0,40734,786.05580,-7.389911,-7.556202,Rural_Akatsi,14.795113,1125.2,-3.1,33.1,15.00,0.62,Sandy,6.169393,0.085267,1.3,0.751354,cassava,0.577964
1,30629,674.33410,-7.736849,-1.051539,Rural_Sokoto,11.374611,1450.7,-3.9,30.6,13.35,0.64,Volcanic,5.676648,0.399684,2.2,1.069865,cassava,0.486302
2,39924,826.53390,-9.926616,0.115156,Rural_Sokoto,11.339692,2208.9,-1.8,28.4,13.30,0.69,Volcanic,5.331993,0.358029,3.4,2.208801,tea,0.649647
3,5754,574.94617,-2.420131,-6.592215,Rural_Kilimani,7.109855,328.8,-5.8,32.2,13.20,0.54,Loamy,5.328150,0.286687,2.4,1.277635,cassava,0.532348
4,14146,886.35300,-3.055434,-7.952609,Rural_Kilimani,55.007656,785.2,-2.5,31.0,14.25,0.72,Sandy,5.721234,0.043190,1.5,0.832614,wheat,0.555076


In [14]:
# Correct crop names
MD_agric_df['Crop_type'].unique()

array(['cassava', 'tea', 'wheat', 'potato', 'banana', 'coffee', 'rice',
       'maize', 'wheat ', 'tea ', 'cassaval', 'wheatn', 'cassava ',
       'teaa'], dtype=object)

In [15]:
def correct_crop_names(crop_name):
    corrections = {
        'cassava ': 'cassava',
        'wheat ': 'wheat',
        'tea ': 'tea',
        'cassaval': 'cassava',
        'wheatn': 'wheat',
        'teaa': 'tea',
    }
    return corrections.get(crop_name, crop_name)

MD_agric_df['Crop_type'] = MD_agric_df['Crop_type'].apply(correct_crop_names)

MD_agric_df['Crop_type'].unique()

array(['cassava', 'tea', 'wheat', 'potato', 'banana', 'coffee', 'rice',
       'maize'], dtype=object)

In [ ]:
# Change negative values in the Elevation column to positive values
MD_agric_df['Elevation'] = MD_agric_df['Elevation'].abs() 

count    5654.000000
mean      637.790709
std       173.573934
min        35.910797
25%       525.611413
50%       663.058585
75%       764.299550
max      1122.252100
Name: Elevation, dtype: float64

### Analysis
